In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# הגדרות טרנספורמציות
transform = transforms.Compose([
    transforms.Resize((100, 100)),  # שינוי גודל אחיד
    transforms.ToTensor(),          # המרת תמונה למטריצה נומרית
])

# נתיב לתיקיית הדאטהסט
dataset_path = "~/fruits-360"  # עדכני את הנתיב בהתאם

# טעינת הדאטהסט
train_dataset = datasets.ImageFolder(root=f"{dataset_path}/Training", transform=transform)
test_dataset = datasets.ImageFolder(root=f"{dataset_path}/Test", transform=transform)

# יצירת DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# בדיקת מספר הקטגוריות
num_classes = len(train_dataset.classes)
print(f"Number of classes: {num_classes}")


Number of classes: 141


In [15]:
# רשימת כל הקטגוריות
class_names = train_dataset.classes

# מיפוי שם קטגוריה לאינדקס
class_to_idx = train_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

# הדפסת המיפוי
print("Class Mapping:", class_to_idx)


Class Mapping: {'Apple 6': 0, 'Apple Braeburn 1': 1, 'Apple Crimson Snow 1': 2, 'Apple Golden 1': 3, 'Apple Golden 2': 4, 'Apple Golden 3': 5, 'Apple Granny Smith 1': 6, 'Apple Pink Lady 1': 7, 'Apple Red 1': 8, 'Apple Red 2': 9, 'Apple Red 3': 10, 'Apple Red Delicious 1': 11, 'Apple Red Yellow 1': 12, 'Apple Red Yellow 2': 13, 'Apple hit 1': 14, 'Apricot 1': 15, 'Avocado 1': 16, 'Avocado ripe 1': 17, 'Banana 1': 18, 'Banana Lady Finger 1': 19, 'Banana Red 1': 20, 'Beetroot 1': 21, 'Blueberry 1': 22, 'Cabbage white 1': 23, 'Cactus fruit 1': 24, 'Cantaloupe 1': 25, 'Cantaloupe 2': 26, 'Carambula 1': 27, 'Carrot 1': 28, 'Cauliflower 1': 29, 'Cherry 1': 30, 'Cherry 2': 31, 'Cherry Rainier 1': 32, 'Cherry Wax Black 1': 33, 'Cherry Wax Red 1': 34, 'Cherry Wax Yellow 1': 35, 'Chestnut 1': 36, 'Clementine 1': 37, 'Cocos 1': 38, 'Corn 1': 39, 'Corn Husk 1': 40, 'Cucumber 1': 41, 'Cucumber 3': 42, 'Cucumber Ripe 1': 43, 'Cucumber Ripe 2': 44, 'Dates 1': 45, 'Eggplant 1': 46, 'Eggplant long 1': 

In [23]:
import torch.nn.functional as F

# פונקציה להמרת תווית אינדקס ל-One-Hot Encoding
def one_hot_encode(labels, num_classes):
    return F.one_hot(labels, num_classes).float()  # מוודאים float למודל

# דוגמה:
labels_tensor = torch.tensor([140])  # נניח שקיבלנו 3 דוגמאות עם התוויות האלו
one_hot_labels = one_hot_encode(labels_tensor, num_classes)
print(one_hot_labels)


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])


2. הגדרת המודל


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 25 * 25, 128)
        self.fc2 = nn.Linear(128, num_classes)  # לא משתמשים ב-Softmax כאן

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # שינוי ל-Sigmoid
        return x



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SmallFruitCNN(nn.Module):
    def __init__(self, num_classes):
        super(SmallFruitCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 25 * 25, 64)  
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + ReLU + MaxPool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + ReLU + MaxPool
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))  # Fully Connected
        x = torch.sigmoid(self.fc2(x))  # Output layer
        return x


In [ ]:
1️⃣ Conv2D (שכבת קונבולוציה)
מוציאה פילטרים (kernels) קטנים שמגלים מאפיינים בתמונה (כמו קצוות וצבעים).
בדוגמה שלנו, Conv2D(3→16, 3x3):
מקבלת תמונה עם 3 ערוצים (RGB).
מפיקה 16 פילטרים בגודל 3x3.
2️⃣ ReLU (Rectified Linear Unit)
פונקציית הפעלה (Activation Function).
הופכת ערכים שליליים ל-0, כדי לשמור על מידע משמעותי.
3️⃣ MaxPooling2D
לוקחת בלוקים של 2x2 פיקסלים ושומרת רק את הערך הכי גבוה.
מקטינה את התמונה (למשל מ-100×100 ל-50×50).
עוזרת למודל להתמקד במידע החשוב.
4️⃣ Flatten
הופכת את התמונה ממטריצה לווקטור.
אם גודל הפלט הוא (32, 25, 25), הוא נהפך לוקטור של 20,000 ערכים.
5️⃣ Fully Connected (FC)
מחברת את כל הנוירונים בשכבה אחת לשנייה.
FC(20,000→64) – לוקח את ה-20,000 ערכים מה-Flatten ומוציא רק 64 ערכים חשובים.
FC(64→num_classes) – מחבר את 64 הערכים ל-כמות הקטגוריות של הפירות.
6️⃣ Sigmoid
הופך את התוצאות לערכים בין 0 ל-1 (כדי לקבל הסתברות לכל פרי).

3. הגדרת פונקציית האובדן והאופטימיזציה


In [9]:
import torch.optim as optim

# יצירת מודל
model = CNN(num_classes=num_classes)

# פונקציית אובדן ואופטימייזר
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


4. אימון המודל


In [14]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [25]:
device = torch.device("cuda:0") # if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda:0


In [26]:
num_epochs = 10  # אפשר להגדיל לפי הצורך
# לולאת אימון עם One-Hot Encoding
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels_one_hot = one_hot_encode(labels, num_classes).to(device)  # המרת תוויות

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels_one_hot)  # השוואה ל-One-Hot
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

print("Training complete!")



RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

5. בדיקת המודל


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels_one_hot = one_hot_encode(labels, num_classes).to(device)

        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)  # המרת One-Hot לאינדקס
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

